In [1]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))
# sys.path

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate

In [3]:
%load_ext autoreload
%autoreload 2
%aimport optiml.queries
%matplotlib inline

## Initialization

In [4]:
# Initialize Snowflake connection to Snowflake
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.queries import SNFLKQuery
qlib = SNFLKQuery(connection, 'KIV', cache_dir)
sdate = '2022-10-01'
edate = '2022-10-31'

/Users/skumar/.snowsql/config
Connecting...


# Total cost breakdown

## Visualization

## Breakdown of total cost by usage type

In [11]:
df = qlib.total_cost_breakdown(sdate, edate)
print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))

Loading data from cache...


In [ ]:
##TODO create a row for totals and print that out separately
## Is there an excel like visualization of this possible?
df_total = df
df_total.sum()

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"},{"type": "pie"}]]
)

fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['dollars'].tolist(),name="Dollars"),row=1,col=1)
fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['credits'].tolist(),name='Credits'),row=1,col=2)

fig.update_layout(
    title={
        'text': "Breakdown of total cost by usage category",
        'y':0.1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'bottom'})
fig.show()

## Analysis
- Compute forms overwhelming majority of the cost
- Cloud services are in aggregate < 10 % of compute so are not an added expense
- Autoclustering used to be on in 2020 but isn't anymore
- Should explore if there are any compute efficiencies to be gained by 
    - Autoclustering
    - Materialized views / caching
    - Search optimization
    - Query optimizations / reducing scans

* Largely used by compute
* In 2020 auto clustering was on but was turned off - why?
* Storage credits at 0 since storage is charged as a flat rate - are numbers consistent?
* Definitely did not use $80K worth of credits in October since total bill is $50K - so something is wrong!


## Breakdown of total cost by warehouse

In [ ]:
df = qlib.cost_by_wh(sdate, edate).round(2)
print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"},{"type": "pie"}]]
)

fig.add_trace(go.Pie(labels=df['warehouse_name'].tolist(), values=df['dollars'].tolist(),name='dollars'),row=1,col=1)
fig.add_trace(go.Pie(labels=df['warehouse_name'].tolist(), values=df['credits'].tolist(),name='credits'),row=1,col=2)

fig.update_layout(
    title={
        'text': "Breakdown of total cost by warehouse",
        'y':0.1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'bottom'})
fig.show()

## Analysis
- Sum of the compute across all the warehouses is 1255.47 credits which is consistent with total cost of compute above. Sanity check to ensure we are not missing any compute or warehouse

In [8]:
df = qlib.test_func('hello')
df

Loading data from cache...


,A,B
0,1,2
1,3,4
